In [ ]:
# === CCC-GARCH in PyMC (joint estimation) ===
import numpy as np
import pandas as pd
import pymc as pm
import aesara
import aesara.tensor as at
import arviz as az

# Inputs expected:
# df:            (T x n) DataFrame of observed series (rows=dates, cols=series ids)
# df_cond_vol:   (T x n) optional conditional std devs to seed sigma^2 at the first step

# ------------- Prepare data -------------
df = df.sort_index()
Y = df.values.astype(np.float64)  # shape (T, n)
dates = df.index
series_names = df.columns
T, n = Y.shape
if T < 3:
    raise ValueError("Need at least 3 observations because σ_t uses X_{t-2}.")

# Optional seed from prefit vols
use_seed_sigma = False
sigma2_seed = None
try:
    if isinstance(df_cond_vol, pd.DataFrame):
        df_cond_vol = df_cond_vol.loc[dates, series_names]
        sigma2_seed = (df_cond_vol.iloc[0].values.astype(np.float64)) ** 2  # (n,)
        use_seed_sigma = True
except NameError:
    pass

Y_shared = aesara.shared(Y)

with pm.Model() as ccc_garch:
    # ---------- Priors per series ----------
    c     = pm.Normal("c",   mu=100.0, sigma=20.0, shape=n)    # c_i
    phi   = pm.Normal("phi", mu=0.0,   sigma=0.5, shape=n)     # φ_i
    omega = pm.TruncatedNormal("omega", mu=100.0, sigma=20.0, lower=1e-8, shape=n)  # ω_i > 0
    alpha = pm.Beta("alpha", alpha=5.0, beta=2.0, shape=n)     # α_i ∈ (0,1)
    beta  = pm.Beta("beta",  alpha=2.0, beta=5.0, shape=n)     # β_i ∈ (0,1)

    # Soft stationarity: α + β < 1
    pm.Potential("stationarity_soft",
        -100.0 * at.sum(at.nnet.relu(alpha + beta - 0.999))
    )

    # ---------- Constant correlation for z_t ----------
    # Cholesky factor of a correlation matrix via LKJ prior (more efficient than dense cov)
    lkj = pm.LKJCholeskyCov.dist(
        n=n,
        eta=2.0,              # mild shrinkage to identity
        sd_dist=pm.HalfNormal.dist(1.0)  # standard deviations; here they’ll be 1.0 in expectation
    )
    L = pm.RandomVariable("L", lkj)      # lower-triangular Cholesky of covariance
    # Because z are standardized, covariance = correlation; using L as 'chol' is appropriate.
    # If you prefer, you can also get the full correlation matrix via:
    # R = pm.Deterministic("R", L @ L.T)

    # ---------- AR(1)-GARCH(1,1) recursion ----------
    # μ_{i,t} = c_i + φ_i X_{i,t-1}
    # σ^2_{i,t} = ω_i + α_i (X_{i,t-1} - c_i - X_{i,t-2})^2 + β_i σ^2_{i,t-1}

    # Initial variance at t=1
    if use_seed_sigma:
        sigma2_init = at.as_tensor_variable(sigma2_seed)  # (n,)
    else:
        denom = at.clip(1.0 - alpha - beta, 1e-6, 1e6)    # avoid division by tiny numbers
        sigma2_init = omega / denom                       # (n,)

    def step(y_tm1, y_tm2, sigma2_tm1, c, phi, omega, alpha, beta):
        mu_t = c + phi * y_tm1
        arch_term = (y_tm1 - c - y_tm2) ** 2
        sigma2_t = omega + alpha * arch_term + beta * sigma2_tm1
        sigma2_t = at.clip(sigma2_t, 1e-12, 1e12)         # numeric safety
        return mu_t, sigma2_t

    y_tm1_seq = Y_shared[1:]   # (T-1, n)
    y_tm2_seq = Y_shared[:-1]  # (T-1, n)

    (mu_seq, sigma2_seq), _ = aesara.scan(
        fn=step,
        sequences=[y_tm1_seq, y_tm2_seq],
        outputs_info=[None, sigma2_init],
        non_sequences=[c, phi, omega, alpha, beta]
    )
    sigma_seq = at.sqrt(sigma2_seq)                       # (T-1, n)
    z_seq = (Y_shared[1:] - mu_seq) / sigma_seq           # (T-1, n)

    # ---------- Likelihood (CCC part) ----------
    # Each row z_t ~ MVN(0, R) with Cholesky L; i.i.d. over time given params.
    pm.MvNormal("z",
                mu=at.zeros(n),
                chol=L,
                observed=z_seq)

    # ---------- Inference ----------
    trace = pm.sample(draws=1500, tune=1500, chains=4, target_accept=0.9, random_seed=123)

    # Optionally save μ_t and σ_t sequences
    pm.Deterministic("mu_seq", mu_seq)
    pm.Deterministic("sigma_seq", sigma_seq)

    ppc = pm.sample_posterior_predictive(trace, var_names=["z"])

# Quick summaries
print(az.summary(trace, var_names=["c","phi","omega","alpha","beta"]))


In [ ]:
# --- Two-step CCC-GARCH in PyMC: estimate only the constant correlation R ---
import numpy as np
import pandas as pd
import pymc as pm
import aesara.tensor as at
import arviz as az

# Inputs you already have:
# df            : (T x n) observed series, rows=dates, cols=series ids
# df_cond_mean  : (T x n) conditional means from your univariate AR(1)
# df_cond_vol   : (T x n) conditional std devs from your univariate GARCH(1,1)

# 1) Align and standardize
df      = df.sort_index()
df_cond_mean = df_cond_mean.loc[df.index, df.columns]
df_cond_vol  = df_cond_vol.loc[df.index, df.columns]

# guard against zero/neg vols (shouldn't happen, but be safe numerically)
eps = 1e-12
S   = np.maximum(df_cond_vol.values.astype(float), eps)          # (T, n)
M   = df_cond_mean.values.astype(float)                          # (T, n)
Y   = df.values.astype(float)                                    # (T, n)

Z   = (Y - M) / S                                                # standardized residuals (T, n)

# If your univariate filters use lags, the first row(s) may be unreliable.
# Common to drop the first 1–2 rows; adjust k as you see fit.
k = 1
Z_use = Z[k:, :]                                                 # (T-k, n)
T_eff, n = Z_use.shape

# 2) PyMC model: z_t ~ MVN(0, R) i.i.d. over t, with R constant (CCC)
with pm.Model() as ccc_step2:
    # Easiest: sample correlation matrix directly (valid & PD)
    R = pm.LKJCorr("R", n=n, eta=2.0)                            # mild shrinkage to I

    # Likelihood on standardized residuals
    pm.MvNormal("z",
                mu=at.zeros(n),
                cov=R,                                           # correlation works as covariance here
                observed=Z_use)

    trace_R = pm.sample(draws=2000, tune=2000, chains=4,
                        target_accept=0.9, random_seed=123)

# 3) Quick summaries
print(az.summary(trace_R, var_names=["R"]))

# Example: posterior mean correlation matrix
R_mean = trace_R.posterior["R"].mean(dim=("chain","draw")).values
R_df = pd.DataFrame(R_mean, index=df.columns, columns=df.columns)
print("\nPosterior mean R:\n", R_df.round(3))

# Optional: compare to the empirical correlation of Z_use
emp_corr = pd.DataFrame(Z_use).corr().values
emp_corr_df = pd.DataFrame(emp_corr, index=df.columns, columns=df.columns)
print("\nEmpirical corr of standardized residuals:\n", emp_corr_df.round(3))


In [ ]:
# Joint CCC-GARCH with time-updating mu_t, sigma_t and constant correlation R
import numpy as np
import pandas as pd
import pymc as pm
import aesara
import aesara.tensor as at
import arviz as az

# --- Inputs ---
# df: (T x n) DataFrame of the observed series (rows=dates, cols=series ids)
# OPTIONAL: df_cond_vol to seed the initial variance (first row used)

# Prepare data
df = df.sort_index()
Y = df.values.astype(np.float64)         # (T, n)
dates = df.index
series = df.columns
T, n = Y.shape
if T < 3:
    raise ValueError("Need at least 3 observations (variance uses X_{t-2}).")

# Optional seeding of initial variance from univariate fits
use_seed_sigma = False
sigma2_seed = None
try:
    if isinstance(df_cond_vol, pd.DataFrame):
        df_cond_vol = df_cond_vol.loc[dates, series]
        sigma2_seed = (df_cond_vol.iloc[0].values.astype(np.float64)) ** 2  # (n,)
        use_seed_sigma = True
except NameError:
    pass

Y_shared = aesara.shared(Y)

with pm.Model() as ccc_joint:
    # ----- Priors (per series) -----
    c     = pm.Normal("c",   mu=100.0, sigma=20.0, shape=n)
    phi   = pm.Normal("phi", mu=0.0,   sigma=0.5,  shape=n)
    omega = pm.TruncatedNormal("omega", mu=100.0, sigma=20.0, lower=1e-8, shape=n)
    alpha = pm.Beta("alpha", alpha=5.0, beta=2.0, shape=n)
    beta  = pm.Beta("beta",  alpha=2.0, beta=5.0, shape=n)

    # Soft stationarity: alpha + beta < 1
    pm.Potential("stationarity_soft",
        -100.0 * at.sum(at.nnet.relu(alpha + beta - 0.999))
    )

    # ----- Constant correlation (CCC) -----
    # Correlation-only Cholesky (unit variances): faster & stable
    L = pm.LKJCorrCholesky("L", n=n, eta=2.0)
    R = pm.Deterministic("R", L @ L.T)

    # ----- AR(1)-GARCH(1,1) recursion to get mu_t and sigma_t -----
    # Initialize sigma^2 at t=1
    if use_seed_sigma:
        sigma2_init = at.as_tensor_variable(sigma2_seed)            # (n,)
    else:
        denom = at.clip(1.0 - alpha - beta, 1e-6, 1e6)
        sigma2_init = omega / denom                                 # (n,)

    def step(y_tm1, y_tm2, sigma2_tm1, c, phi, omega, alpha, beta):
        mu_t = c + phi * y_tm1                                      # (n,)
        arch = (y_tm1 - c - y_tm2) ** 2                             # (n,)
        sigma2_t = omega + alpha * arch + beta * sigma2_tm1         # (n,)
        sigma2_t = at.clip(sigma2_t, 1e-12, 1e12)                   # guard
        return mu_t, sigma2_t

    y_tm1_seq = Y_shared[1:]     # (T-1, n) -> uses X_{t-1}
    y_tm2_seq = Y_shared[:-1]    # (T-1, n) -> uses X_{t-2}

    (mu_seq, sigma2_seq), _ = aesara.scan(
        fn=step,
        sequences=[y_tm1_seq, y_tm2_seq],
        outputs_info=[None, sigma2_init],
        non_sequences=[c, phi, omega, alpha, beta]
    )
    sigma_seq = at.sqrt(sigma2_seq)                                  # (T-1, n)

    # Expose for inspection
    pm.Deterministic("mu_seq", mu_seq)
    pm.Deterministic("sigma_seq", sigma_seq)

    # ----- Likelihood: X_t | F_{t-1} ~ N(mu_t, D_t R D_t) -----
    # Build per-time log-likelihood with time-varying chol_t = diag(sigma_t) @ L
    def ll_step(mu_t, sigma_t, y_t, L):
        chol_t = at.diag(sigma_t) @ L                                # (n,n)
        # vector MVN with time-specific covariance
        logp_t = pm.logp(pm.MvNormal.dist(mu=mu_t, chol=chol_t), y_t)
        return logp_t

    logp_seq, _ = aesara.scan(
        fn=ll_step,
        sequences=[mu_seq, sigma_seq, Y_shared[1:]],
        non_sequences=[L]
    )
    pm.Potential("likelihood", at.sum(logp_seq))

    # ----- Inference -----
    trace = pm.sample(
        draws=1500, tune=1500, chains=4,
        target_accept=0.9, random_seed=123
    )

    # (Optional) posterior predictive for X given parameters would require forward simulation;
    # PPC on this likelihood is not automatic since cov changes by t.

# Quick summaries
print(az.summary(trace, var_names=["c","phi","omega","alpha","beta"]))


In [ ]:
# CCC

import numpy as np
import pandas as pd
from scipy.linalg import cholesky, solve_triangular

# -----------------------------
# Utilities
# -----------------------------

def _nearest_correlation(A, tol=1e-8, max_iters=100):
    """
    Higham (2002) projection to the nearest correlation matrix.
    Ensures symmetric PD with unit diagonal.
    """
    # Symmetrize
    A = (A + A.T) / 2.0
    n = A.shape[0]
    X = A.copy()
    Y = A.copy()
    delta_S = np.zeros_like(A)

    for _ in range(max_iters):
        R = X - delta_S
        # PSD projection
        eigvals, eigvecs = np.linalg.eigh(R)
        eigvals_clipped = np.clip(eigvals, tol, None)
        X = (eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T)
        X = (X + X.T) / 2.0  # re-sym
        delta_S = X - R
        # Reset diagonal to 1
        np.fill_diagonal(X, 1.0)
        # Convergence check
        if np.linalg.norm(X - Y, ord='fro') < 1e-10:
            break
        Y = X.copy()
    # Final symmetrize
    X = (X + X.T) / 2.0
    np.fill_diagonal(X, 1.0)
    return X

def _pairwise_complete_corr(Z: pd.DataFrame) -> pd.DataFrame:
    """
    Correlation matrix using pairwise complete observations.
    """
    # Pandas corr uses pairwise complete obs by default
    R = Z.corr()
    # Fill diag with 1 exactly
    np.fill_diagonal(R.values, 1.0)
    return R

def _row_mask_complete(*dfs):
    """
    Mask of rows where ALL provided dataframes have no NaNs across ALL columns.
    """
    mask = pd.Series(True, index=dfs[0].index)
    for d in dfs:
        mask &= d.notna().all(axis=1)
    return mask

# -----------------------------
# CCC-GARCH core
# -----------------------------

def estimate_ccc_from_univariates(df, df_cond_mean, df_cond_vol, shrink_to_identity=0.0):
    """
    Estimate CCC-GARCH correlation matrix R using standardized residuals z_{i,t}.
    Inputs:
        df            : DataFrame of raw series (T x N)
        df_cond_mean  : DataFrame of conditional means X_{i,t} (T x N)
        df_cond_vol   : DataFrame of conditional std devs sigma_{i,t} (T x N)
        shrink_to_identity: float in [0,1]. Optional shrinkage toward I.
                            R_hat <- (1 - s)*R_sample + s*I
    Returns:
        results dict with:
            - 'R': pd.DataFrame (N x N), PD correlation matrix
            - 'Z': standardized residuals DataFrame (T x N)
            - 'Eps': residuals DataFrame (T x N)
            - 'loglik': float (quasi log-likelihood, in-sample, using rows with complete data)
            - 'H_t': dict {timestamp: (N x N) covariance matrix as pd.DataFrame}
    """
    # Align and sanity-check
    df, df_cond_mean, df_cond_vol = df.align(df_cond_mean, join='inner', axis=None)
    df, df_cond_vol = df.align(df_cond_vol, join='inner', axis=None)

    # Residuals and standardized residuals
    Eps = df - df_cond_mean
    Z = Eps / df_cond_vol

    # Initial correlation estimate (pairwise complete)
    R_sample = _pairwise_complete_corr(Z)

    # Optional shrinkage to identity for robustness
    if not (0.0 <= shrink_to_identity <= 1.0):
        raise ValueError("shrink_to_identity must be in [0,1].")
    if shrink_to_identity > 0.0:
        I = np.eye(R_sample.shape[0])
        R_shrunk = (1.0 - shrink_to_identity) * R_sample.values + shrink_to_identity * I
        R = pd.DataFrame(R_shrunk, index=R_sample.index, columns=R_sample.columns)
    else:
        R = R_sample.copy()

    # Project to nearest valid correlation matrix
    R_pd = _nearest_correlation(R.values)
    R = pd.DataFrame(R_pd, index=R.index, columns=R.columns)

    # Build per-time covariance matrices H_t = D_t R D_t
    # Use only rows where df_cond_vol has all sigmas present (positive)
    valid_rows = df_cond_vol.notna().all(axis=1)
    # Guard against non-positive sigmas
    positive_rows = (df_cond_vol > 0).all(axis=1)
    row_mask = valid_rows & positive_rows

    H_t = {}
    cols = df.columns.tolist()
    R_np = R.values
    for t, row in df_cond_vol[row_mask].iterrows():
        D = np.diag(row.values)  # diag of sigmas at time t
        H = D @ R_np @ D
        H_t[t] = pd.DataFrame(H, index=cols, columns=cols)

    # Quasi log-likelihood using rows where we have complete data across df/Eps/sigmas
    complete_mask = _row_mask_complete(df, df_cond_mean, df_cond_vol)
    Z_complete = Z.loc[complete_mask]
    Sigma_complete = df_cond_vol.loc[complete_mask]

    # Log-likelihood for multivariate normal with mean = cond_mean and cov H_t
    # l_t = -0.5 [ N log(2π) + log|H_t| + e_t' H_t^{-1} e_t ]
    # Efficiently via Cholesky of H_t
    const = -0.5 * Z_complete.shape[1] * np.log(2.0 * np.pi)
    total_ll = 0.0
    for t in Z_complete.index:
        d = Sigma_complete.loc[t].values
        # H_t = D R D -> log|H_t| = 2*sum(log d_i) + log|R|
        # And e'H^{-1}e = (D^{-1} e)' R^{-1} (D^{-1} e) = z' R^{-1} z
        # So we can reuse constant R across t
        # Precompute Cholesky of R once
        # (we will compute once outside the loop for efficiency)
        pass
    # Precompute parts for all t
    dlogs = np.log(Sigma_complete.values).sum(axis=1)  # sum log sigmas at each t
    # Cholesky of R
    L = cholesky(R_np, lower=True, check_finite=True)
    log_det_R = 2.0 * np.log(np.diag(L)).sum()
    # Solve for z via L
    for i, t in enumerate(Z_complete.index):
        z = Z_complete.loc[t].values
        # Solve L y = z => y = L^{-1} z
        y = solve_triangular(L, z, lower=True, check_finite=False)
        quad = y @ y  # ||y||^2
        logdet_Ht = 2.0 * dlogs[i] + log_det_R
        lt = const - 0.5 * (logdet_Ht + quad)
        total_ll += lt

    return {
        "R": R,
        "Z": Z,
        "Eps": Eps,
        "loglik": float(total_ll),
        "H_t": H_t,
    }

def ccc_covariance_path(df_cond_vol, R):
    """
    Given per-time sigmas and a fixed correlation matrix R,
    return covariance matrices H_t = D_t R D_t for all rows with complete positive sigmas.
    """
    R_np = np.asarray(R)
    cols = df_cond_vol.columns.tolist()
    valid_rows = df_cond_vol.notna().all(axis=1) & (df_cond_vol > 0).all(axis=1)
    H_t = {}
    for t, row in df_cond_vol[valid_rows].iterrows():
        D = np.diag(row.values)
        H = D @ R_np @ D
        H_t[t] = pd.DataFrame(H, index=cols, columns=cols)
    return H_t

def ccc_one_step_forecast(sigma_next: pd.Series, R: pd.DataFrame):
    """
    One-step-ahead covariance forecast: H_{t+1|t} = D_{t+1} R D_{t+1}.
    Inputs:
        sigma_next : pd.Series indexed by series IDs with next-step sigmas (>0)
        R          : correlation matrix (pd.DataFrame)
    Returns:
        H_next     : DataFrame covariance matrix
    """
    sigma_next = sigma_next.dropna()
    ids = R.index.intersection(sigma_next.index)
    D = np.diag(sigma_next.loc[ids].values)
    R_sub = R.loc[ids, ids].values
    H = D @ R_sub @ D
    return pd.DataFrame(H, index=ids, columns=ids)

# -----------------------------
# Example usage
# -----------------------------
# Assuming you already have df, df_cond_mean, df_cond_vol as described:
# results = estimate_ccc_from_univariates(df, df_cond_mean, df_cond_vol, shrink_to_identity=0.05)
# R_hat = results["R"]          # constant correlation matrix
# loglik = results["loglik"]    # quasi log-likelihood (in-sample)
# H_t = results["H_t"]          # dict of per-date covariance matrices
# Z = results["Z"]              # standardized residuals
# Eps = results["Eps"]          # raw residuals
#
# # Optional: compute the entire covariance path again or forecast one-step ahead
# H_path = ccc_covariance_path(df_cond_vol, R_hat)
# # Suppose you’ve computed/forecasted sigma for t+1 in a Series:
# # H_next = ccc_one_step_forecast(sigma_next_series, R_hat)


In [ ]:
# DCC

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.linalg import cholesky, solve_triangular

# ==========================
# Helpers
# ==========================

def nearest_correlation(A, tol=1e-12, max_iters=100):
    """
    Higham (2002) projection to the nearest correlation matrix.
    Ensures symmetric PSD with unit diagonal. Adds tiny jitter if needed.
    """
    A = (A + A.T) / 2.0
    n = A.shape[0]
    X = A.copy()
    Y = A.copy()
    delta_S = np.zeros_like(A)

    for _ in range(max_iters):
        R = X - delta_S
        # PSD projection
        eigvals, eigvecs = np.linalg.eigh(R)
        eigvals = np.clip(eigvals, tol, None)
        X = (eigvecs @ np.diag(eigvals) @ eigvecs.T)
        X = (X + X.T) / 2.0
        delta_S = X - R
        np.fill_diagonal(X, 1.0)
        if np.linalg.norm(X - Y, ord='fro') < 1e-10:
            break
        Y = X.copy()
    X = (X + X.T) / 2.0
    np.fill_diagonal(X, 1.0)
    return X

def softplus(x):
    # stable softplus
    return np.log1p(np.exp(-np.abs(x))) + np.maximum(x, 0)

def ab_from_uv(u, v):
    """
    Reparameterization that enforces a>=0, b>=0, a+b<1:
        a = exp(u) / (1 + exp(u) + exp(v))
        b = exp(v) / (1 + exp(u) + exp(v))
    """
    eu, ev = np.exp(u), np.exp(v)
    denom = 1.0 + eu + ev
    a = eu / denom
    b = ev / denom
    return a, b

def loglike_dcc(z, a, b, project=True):
    """
    DCC(1,1) log-likelihood contribution (multivariate part only).
    z: (T x N) standardized residuals with no NaNs
    Returns: total loglik, arrays of Qt (T x N x N) and Rt, and last Qt
    """
    T, N = z.shape
    # Unconditional correlation of z
    Qbar = np.corrcoef(z.T)
    np.fill_diagonal(Qbar, 1.0)

    # Initialize Q0 as Qbar
    Qt = np.zeros((T, N, N))
    Rt = np.zeros_like(Qt)

    Q_prev = Qbar.copy()

    # Precompute to speed up
    const = -0.5 * N * np.log(2.0 * np.pi)

    total_ll = 0.0
    for t in range(T):
        # DCC recursion
        zz = np.outer(z[t - 1], z[t - 1]) if t > 0 else Qbar  # use Qbar for t=0
        Q_t = (1 - a - b) * Qbar + a * zz + b * Q_prev

        # Scale to R_t
        d = np.sqrt(np.clip(np.diag(Q_t), 1e-12, None))
        invd = 1.0 / d
        R_t = (invd[:, None] * Q_t) * invd[None, :]

        # Numerical safety: project if needed
        try:
            L = cholesky(R_t, lower=True, check_finite=True)
        except Exception:
            if not project:
                # add tiny jitter on the diagonal scaling to avoid failure
                jitter = 1e-10
                R_t = R_t + np.eye(N) * jitter
                L = cholesky(R_t, lower=True, check_finite=False)
            else:
                R_t = nearest_correlation(R_t)
                L = cholesky(R_t, lower=True, check_finite=False)

        # contribution: -0.5*(log|R_t| + z_t' R_t^{-1} z_t) + const
        log_det_Rt = 2.0 * np.log(np.diag(L)).sum()
        y = solve_triangular(L, z[t], lower=True, check_finite=False)
        quad = y @ y
        lt = const - 0.5 * (log_det_Rt + quad)
        total_ll += lt

        Qt[t] = Q_t
        Rt[t] = R_t
        Q_prev = Q_t

    return float(total_ll), Qt, Rt, Q_prev, Qbar

# ==========================
# DCC API
# ==========================

def fit_dcc_from_univariates(df, df_cond_mean, df_cond_vol,
                             u0=np.log(0.05/0.90), v0=np.log(0.93/0.90),
                             project=True, verbose=False):
    """
    Fit Engle's DCC(1,1) after univariate AR-GARCH:
      z_{i,t} = (y_{i,t} - X_{i,t}) / sigma_{i,t}

    Inputs:
      df, df_cond_mean, df_cond_vol : DataFrames (aligned by date x series)
      u0, v0 : initial values for unconstrained parameters (map -> a,b)
      project: if True, project Rt to nearest correlation on failures
      verbose: print optimizer status

    Returns dict with:
      'params' : {'a':..., 'b':...}
      'loglik' : total DCC log-likelihood (multivariate part)
      'Z'      : standardized residuals (complete-case)
      'Qt', 'Rt': np arrays (T_eff x N x N)
      'Qbar'   : unconditional correlation of Z
      'H_t'    : dict mapping timestamp -> covariance matrix (DataFrame)
      'complete_index': index used in estimation
    """
    # Align and compute standardized residuals
    df, df_cond_mean, df_cond_vol = df.align(df_cond_mean, join='inner', axis=None)
    df, df_cond_vol = df.align(df_cond_vol, join='inner', axis=None)
    eps = df - df_cond_mean
    z = eps / df_cond_vol

    # DCC requires complete cases across series during estimation
    mask = z.notna().all(axis=1) & (df_cond_vol > 0).all(axis=1)
    Z = z.loc[mask].values
    idx = z.index[mask]
    cols = z.columns.tolist()

    if Z.shape[0] < 10:
        raise ValueError("Not enough complete observations for DCC estimation.")

    # Objective = NEGATIVE log-likelihood (to minimize)
    def obj(theta):
        u, v = theta
        a, b = ab_from_uv(u, v)
        # Penalize extreme sums near 1 to help numerics
        penalty = 1e4 * np.maximum(a + b - 0.999, 0.0)**2
        ll, *_ = loglike_dcc(Z, a, b, project=project)
        return -(ll) + penalty

    theta0 = np.array([u0, v0])  # sensible defaults: a~0.05, b~0.93
    res = minimize(obj, theta0, method="L-BFGS-B")

    if verbose:
        print(res)

    u_hat, v_hat = res.x
    a_hat, b_hat = ab_from_uv(u_hat, v_hat)
    ll, Qt, Rt, Q_last, Qbar = loglike_dcc(Z, a_hat, b_hat, project=project)

    # Build H_t = D_t R_t D_t over the *same* complete index
    Sigma = df_cond_vol.loc[idx].values
    H_t = {}
    for t, ts in enumerate(idx):
        D = np.diag(Sigma[t])
        H = D @ Rt[t] @ D
        H_t[ts] = pd.DataFrame(H, index=cols, columns=cols)

    return {
        "params": {"a": float(a_hat), "b": float(b_hat)},
        "loglik": float(ll),
        "Z": pd.DataFrame(Z, index=idx, columns=cols),
        "Qt": Qt,            # numpy array
        "Rt": Rt,            # numpy array
        "Q_last": Q_last,    # numpy array (for forecasting)
        "Qbar": Qbar,        # numpy array
        "H_t": H_t,          # dict of DataFrames
        "complete_index": idx,
        "columns": cols,
    }

def dcc_one_step_forecast(sigma_next: pd.Series,
                          last_z: pd.Series,
                          Q_last: np.ndarray,
                          Qbar: np.ndarray,
                          a: float, b: float) -> pd.DataFrame:
    """
    One-step-ahead forecast under DCC(1,1):
      Q_{t+1} = (1-a-b)Qbar + a z_t z_t' + b Q_t
      R_{t+1} = scale(Q_{t+1})
      H_{t+1} = D_{t+1} R_{t+1} D_{t+1}

    Inputs must share the same index/order of series IDs.
    """
    ids = sigma_next.index.intersection(last_z.index)
    sigma_next = sigma_next.loc[ids]
    z_t = last_z.loc[ids].values
    # Subset Q_last/Qbar in the same order
    # (Assumes original order equals 'ids'. If not, reorder externally.)
    n = len(ids)
    Q_t = Q_last
    if Q_t.shape[0] != n:
        raise ValueError("Shape mismatch: reorder Q_last/Qbar to match series ID order.")

    Q_next = (1 - a - b) * Qbar + a * np.outer(z_t, z_t) + b * Q_t
    d = np.sqrt(np.clip(np.diag(Q_next), 1e-12, None))
    invd = 1.0 / d
    R_next = (invd[:, None] * Q_next) * invd[None, :]
    # Safety
    try:
        cholesky(R_next, lower=True, check_finite=True)
    except Exception:
        R_next = nearest_correlation(R_next)

    D_next = np.diag(sigma_next.values)
    H_next = D_next @ R_next @ D_next
    return pd.DataFrame(H_next, index=ids, columns=ids)

# ==========================
# Example usage
# ==========================
# Assume you already have:
#   df            : raw series (T x N)
#   df_cond_mean  : conditional means X_{i,t} (T x N)
#   df_cond_vol   : conditional std devs sigma_{i,t} (T x N)
#
# results = fit_dcc_from_univariates(df, df_cond_mean, df_cond_vol, verbose=True)
# print("Estimated (a, b):", results["params"])
# R_path = results["Rt"]     # time-varying correlations
# H_t    = results["H_t"]    # dict of covariances by timestamp
#
# # Forecast example:
# last_idx = results["complete_index"][-1]
# last_z   = results["Z"].iloc[-1]                      # z_t
# Q_last   = results["Q_last"]                          # Q_t
# Qbar     = results["Qbar"]
# a, b     = results["params"]["a"], results["params"]["b"]
# # Provide sigma_{t+1} as a Series (same IDs, positive)
# # sigma_next = pd.Series(..., index=df.columns)
# # H_next = dcc_one_step_forecast(sigma_next, last_z, Q_last, Qbar, a, b)
